In [20]:
import glob
import numpy as np
import pandas as pd
import ole

path = ('./이력서/*.hwp')
totaldata = []
for fname in glob.glob(path):
       with ole.open(fname) as f:
            totaldata.append(f.get_stream('PrvText').read().decode('utf-16le').split('><'))

In [21]:
data_columns = [
      '이름', 
      '생년월일',
      '주소',
      '번호',
      '이메일',
      '개인정보 데이터원본', ## 확인용 데이터 원본
      '대학교',
      '전공',
      '학점', 
      '졸업구분', 
      '소재지', 
      '학교정보 데이터원본', ## 확인용 데이터 원본
      'TOEIC', 
      '토익 데이터원본', ## 확인용 데이터 원본
      '자격증',
      '자격증 데이터원본', ## 확인용 데이터 원본
      '교육/활동/수상',
      '교육/활동/수상 데이터원본', ## 확인용 데이터 원본
      '군필여부', 
      '군별', 
      '계급',
      '복무기간', 
      '군필 데이터원본', ## 확인용 데이터 원본
      '컴퓨터언어능력',
      '컴퓨터언어능력 데이터원본', ## 확인용 데이터 원본
     ]

In [22]:
df = pd.DataFrame(np.zeros((100, 25)), columns=data_columns)
df = df.T

In [23]:
## 최종
def privacyinfo(splitdata):
    result = []
    
    privacyIndexData = [
    '성          명',
    '생  년  월  일',
    '현    주    소',
    '연    락    처',
    '이    메    일'
     ]
    for i in range(len(splitdata)):
        for j in range(len(privacyIndexData)):
            if splitdata[i] == '이    메    일':
                emailSplitFindIndex = splitdata[i+1].find('>')
                result.append(splitdata[i+1][:emailSplitFindIndex])
                break
            if splitdata[i] == privacyIndexData[j]:
                result.append(splitdata[i+1])
    return result

def schoolinfo(splitdata):
    result = []

    schoolIndexData = ['학교명', '전공', '성적', '졸업구분', '소재지']
    for i in range(len(splitdata)):
        for j in range(len(schoolIndexData)):
            if splitdata[i] == '성적':
                if splitdata[i+5] != '':
                    score = float(splitdata[i+5].replace('/4.5', ''))
                    if score >= 4.0:
                        result.append(10)
                        break
                    elif score >= 3.7:
                        result.append(9)
                        break
                    elif score >= 3.5:
                        result.append(8)
                        break
                    elif score >= 3.0:
                        result.append(7)
                        break
                    else:
                        result.append(6)
                        break
                else :
                    result.append(0) ## 수정사항 : 학점이 들어가지 않은 데이터는 0점으로 준다
                    break

            if splitdata[i] == '소재지':
                idx = splitdata[i+5].find('>')
                result.append(splitdata[i+5][:idx])
                break
                                                         
            if splitdata[i] == schoolIndexData[j]:
                result.append(splitdata[i+5])
    return result

def toeicinfo(splitdata):
    '''
    900점 이상 10점
    900점 ~ 800점 9.5점
    800점 ~ 700점 9점
    700점 ~ 600점 8.5점
    600점 미만 4점
    '''
    result = []
    for i in range(len(splitdata)):
        if splitdata[i] == 'TOEIC':
            if splitdata[i+5] != '': ##수정사항 : 만약 점수가 공백이 아니면 수행
                try:
                    score = int(splitdata[i+5].replace('점', ''))
                except ValueError:
                    continue
                result.append(score)
                if score >= 900:
                    result.append(10)
                    break
                elif score >= 800:
                    result.append(9.5)
                    break
                elif score >= 700:
                    result.append(9)
                    break
                elif score >= 600:
                    result.append(8.5)
                    break
                else:
                    result.append(4)
                    break
            else: ## 수정사항 공백이면 토익점수와 변환점수를 0,0으로 append
                result.append(0)
                result.append(0)
                break
    return result

    
def certificateinfo(splitdata):
    '''
    기사자격증 - 4점
    그외 기타 자격증 - 1점
    만점 - 5점
    '''
    result = []
    
    idx_start = splitdata.index('점수')
    idx_end = splitdata.index('기간')
    result_origin = splitdata[idx_start+1:idx_end]
    
    del result_origin[3]
    result_origin[-1] = result_origin[-1][:result_origin[-1].find('>')]
    
    result.append(result_origin)
    
    score = 0
    certificateName = []
    for i in range(0, len(result_origin), 3):
        ## 수정사항 : 공백 부분도 1점으로 들어가서 점수 안 들어가게 넘기기
        if result_origin[i] == '':
            continue
        else:
            if result_origin[i].find('기사') != -1:
                # print(result_origin[i])
                certificateName.append(result_origin[i])
                score += 4
            else:
                # print(result_origin[i])
                certificateName.append(result_origin[i])
                score += 1

    result.append(certificateName)
    if score > 5:
        score = 5
    result.append(score)
    
    return result    

def educationinfo(splitdata):
    '''
    교육/활동/수상 - 1회당 2점 (가산점)
    최대 5점
    '''
    result = []
        
    교육index = [
        '군별', 
        '계급', 
        '복무기간'
    ]
    활동 = []
    for i in 교육index:
        if splitdata[splitdata.index(i)-1]:
            활동.append(splitdata[splitdata.index(i)-1])
    
    score = len(활동) * 2
    
    if score > 5:
        score = 5
        
    result.append(활동)
    result.append(score)
        
    return result

def militaryinfo(splitdata):
    result = []
    군복무index = [
        '군필여부',
        '군별',
        '계급',
        '복무기간'
    ]
    
    for i in range(len(splitdata)):
        for j in range(len(군복무index)):
            if splitdata[i] == '복무기간':
                idx = splitdata[i+1].find('>')
                result.append(splitdata[i+1][:idx])
                break
            if splitdata[i] == 군복무index[j]:
                result.append(splitdata[i+1])
                
    return result

def languageinfo(splitdata):
    '''
    상 - 2점
    중 - 1.5점
    하 - 1점
    '''
    result = []
    언어index = ['R', 'Python', 'Javascript', 'SQL', 'HTML/CSS']
    for i in range(len(splitdata)):
        for j in range(len(언어index)):
            if splitdata[i] == 'HTML/CSS':
                idx = splitdata[i+5].find('>')
                result.append(splitdata[i+5][:idx])
                break
            if splitdata[i] == 언어index[j]:
                result.append(splitdata[i+5])
    
    score = 0
    score += result.count('상') * 2
    score += result.count('중') * 1.5
    score += result.count('하') * 1
    
    result.append(score)
                
    return result

functions = [
    privacyinfo, 
    schoolinfo, 
    toeicinfo, 
    certificateinfo, 
    educationinfo,
    militaryinfo,
    languageinfo
]

In [24]:
idx = 0
for data_ in totaldata:
    origin_data = []
    for function in functions:
        origin_data.append(function(data_))
    try:
        df[idx] = [
        origin_data[0][0], # 이름
        origin_data[0][1], # 생년월일
        origin_data[0][2], # 주소
        origin_data[0][3], # 번호
        origin_data[0][4], # 이메일
        origin_data[0],    ## 확인용 데이터 원본
        origin_data[1][0], # 학교
        origin_data[1][1], # 전공
        origin_data[1][2], # 학점(10점 만점)
        origin_data[1][3], # 졸업구분
        origin_data[1][4], # 소재지 
        origin_data[1],    ## 확인용 데이터 원본
        origin_data[2][1], # 토익(10점 만점) 
        origin_data[2],    ## 확인용 데이터 원본
        origin_data[3][2], # 자격증 (10점 만점) 
        origin_data[3],    ## 확인용 데이터 원본
        origin_data[4][1], # 교육/활동/수상 (5점 만점) 
        origin_data[4][0], ## 확인용 데이터 원본
        origin_data[5][0], # 군필여부 
        origin_data[5][1], # 군별
        origin_data[5][2], # 계급
        origin_data[5][3], # 복무기간 
        origin_data[5],    ## 확인용 데이터 원본
        origin_data[6][-1],# 컴퓨터언어능력 5점 만점
        origin_data[6],    ## 확인용 데이터 원본
    ]
    except IndexError:
        df[idx] = np.nan
        continue
    finally:
        idx += 1
        
df

0    \
이름                                                            홍길동   
생년월일                                                 1993년 8월 13일   
주소                                                    제주특별자치도 제주시   
번호                                                  010-2233-1478   
이메일                                               ab8ab@naver.com   
개인정보 데이터원본      [홍길동, 1993년 8월 13일, 제주특별자치도 제주시, 010-2233-1478...   
대학교                                                         건국대학교   
전공                                                         컴퓨터공학과   
학점                                                             10   
졸업구분                                                           졸업   
소재지                                                            서울   
학교정보 데이터원본                            [건국대학교, 컴퓨터공학과, 10, 졸업, 서울]   
TOEIC                                                           9   
토익 데이터원본                                                 [725, 9]   
자격증                                                             5   
자격증 데이터원본       [[정보처리기사, 한국산업인력공단, 19.05.22, 한국사능력검정 1급, 국사편찬...   
교육/활동/수상                                                        2   
교육/활동/수상 데이터원본                                    [빅데이터 전략 마에스트로]   
군필여부                                                           군필   
군별                                                             육군   
계급                                                             병장   
복무기간                                                  15.02~16.11   
군필 데이터원본                                [군필, 육군, 병장, 15.02~16.11]   
컴퓨터언어능력                                                         7   
컴퓨터언어능력 데이터원본                                [상, 중, 중, 하, 하, 7.0]   

                                                              1    \
이름                                                            박순혁   
생년월일                                                 1995년 4월 14일   
주소                                                      서울특별시 강남구   
번호                                                  010-0729-1959   
이메일                                              sw0729@naver.com   
개인정보 데이터원본      [박순혁, 1995년 4월 14일, 서울특별시 강남구, 010-0729-1959, ...   
대학교                                                         서울대학교   
전공                                                         독어독문학과   
학점                                                             10   
졸업구분                                                           졸업   
소재지                                                            서울   
학교정보 데이터원본                            [서울대학교, 독어독문학과, 10, 졸업, 서울]   
TOEIC                                                          10   
토익 데이터원본                                                [980, 10]   
자격증                                                             2   
자격증 데이터원본       [[독일어 능력 시험, 주한독일문화원, 18.10.16, 한국사능력검정시험1급, 국...   
교육/활동/수상                                                        2   
교육/활동/수상 데이터원본                                        [빅데이터 경진대회]   
군필여부                                                           군필   
군별                                                             육군   
계급                                                             병장   
복무기간                                                  15.06~17.03   
군필 데이터원본                                [군필, 육군, 병장, 15.06~17.03]   
컴퓨터언어능력                                                       7.5   
컴퓨터언어능력 데이터원본                                [중, 중, 중, 중, 중, 7.5]   

                                                              2    \
이름                                                            정화진   
생년월일                                                 1992년 5월 17일   
주소                                                        경기도 과천시   
번호                                                  010-9491-2351   
이메일                                           eleghkwld@naver.com   
개인정보 데이터원본      [정화진, 1992년 5월 17일, 경기도 과천시, 010-9491-2351

In [25]:
df = df.T

### pandas 저장하기

#### excel 파일로 저장 , 읽기

In [26]:
import pandas as pd
# excel 파일로 저장
df.to_excel('pandas_e.xlsx')
# 엑셀파일 읽기
df = pd.read_excel('pandas_e.xlsx')

#### csv 파일로 저장 , 읽기

In [10]:
# csv 파일로 저장
# 한글이 깨지는 경우가 있기 때문에 encoding 추가
df.to_csv('pandas_c.csv', encoding = 'utf-8-sig')
# csv 파일 읽기
df = pd.read_csv('pandas_c.csv')

###  xlsxwriter 저장하기

In [27]:
import ole

print(dir(ole))

with ole.open('이력서/이력서_1.hwp') as f:
    print(f.list_streams())
    print('=' * 40)
    data = f.get_stream('PrvText').read().decode('utf-16le')
    print(data)

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'constants', 'file', 'open', 'storage', 'stream', 'structures', 'utils']
('FileHeader', 'DocInfo', '\x05HwpSummaryInformation', 'PrvImage', 'PrvText', 'Scripts/JScriptVersion', 'Scripts/DefaultJScript', 'DocOptions/_LinkDoc', 'BodyText/Section0')
이  력  서
<인적사항><성          명><홍길동><생  년  월  일><1993년 8월 13일><현    주    소><제주특별자치도 제주시><연    락    처><010-2233-1478><이    메    일><ab8ab@naver.com>

<학력사항><학교명><전공><성적><졸업구분><소재지><건국대학교><컴퓨터공학과><4.1/4.5><졸업><서울>

<자격면허><자격면허증명><시행기관><취득년월><TOEIC><점수><정보처리기사><한국산업인력공단><19.05.22><725점><한국사능력검정 1급><국사편찬위원회><19.08.23>

<교육 / 활동 / 수상><기간><활동 내용><병역사항><군필여부><군필><19.11 ~ 20.03><빅데이터 전략 마에스트로><군별><육군><><><계급><병장><><><복무기간><15.02~16.11>

<컴퓨터 언어   능력 (상/중/하)><R><Python><Javascript><SQL><HTML/CSS><상><중><중><하><하>

위의 내용은 사실과 틀림없음을 확인합니다.

2020년 5월 11일 성명 : 홍길동



In [28]:
splitData = data.split('><')
splitData

['이  력  서\r\n<인적사항',
 '성          명',
 '홍길동',
 '생  년  월  일',
 '1993년 8월 13일',
 '현    주    소',
 '제주특별자치도 제주시',
 '연    락    처',
 '010-2233-1478',
 '이    메    일',
 'ab8ab@naver.com>\r\n\r\n<학력사항',
 '학교명',
 '전공',
 '성적',
 '졸업구분',
 '소재지',
 '건국대학교',
 '컴퓨터공학과',
 '4.1/4.5',
 '졸업',
 '서울>\r\n\r\n<자격면허',
 '자격면허증명',
 '시행기관',
 '취득년월',
 'TOEIC',
 '점수',
 '정보처리기사',
 '한국산업인력공단',
 '19.05.22',
 '725점',
 '한국사능력검정 1급',
 '국사편찬위원회',
 '19.08.23>\r\n\r\n<교육 / 활동 / 수상',
 '기간',
 '활동 내용',
 '병역사항',
 '군필여부',
 '군필',
 '19.11 ~ 20.03',
 '빅데이터 전략 마에스트로',
 '군별',
 '육군',
 '',
 '',
 '계급',
 '병장',
 '',
 '',
 '복무기간',
 '15.02~16.11>\r\n\r\n<컴퓨터 언어   능력 (상/중/하)',
 'R',
 'Python',
 'Javascript',
 'SQL',
 'HTML/CSS',
 '상',
 '중',
 '중',
 '하',
 '하>\r\n\r\n위의 내용은 사실과 틀림없음을 확인합니다.\r\n\r\n2020년 5월 11일 성명 : 홍길동\r\n']

In [29]:
total = []
for function in functions:
    total.extend(function(splitData))

In [30]:
total

['홍길동',
 '1993년 8월 13일',
 '제주특별자치도 제주시',
 '010-2233-1478',
 'ab8ab@naver.com',
 '건국대학교',
 '컴퓨터공학과',
 10,
 '졸업',
 '서울',
 725,
 9,
 ['정보처리기사', '한국산업인력공단', '19.05.22', '한국사능력검정 1급', '국사편찬위원회', '19.08.23'],
 ['정보처리기사', '한국사능력검정 1급'],
 5,
 ['빅데이터 전략 마에스트로'],
 2,
 '군필',
 '육군',
 '병장',
 '15.02~16.11',
 '상',
 '중',
 '중',
 '하',
 '하',
 7.0]

### 1. 확인용 데이터 원본 제외하고 출력하기  

In [31]:
## ## 확인용 데이터 원본 제외하고 출력하기 

def privacyinfo(splitdata):
    result = []
    privacyIndexData = [
    '성          명',
    '생  년  월  일',
    '현    주    소',
    '연    락    처',
    '이    메    일'
     ]
    for i in range(len(splitdata)):
        for j in range(len(privacyIndexData)):
            if splitdata[i] == '이    메    일':
                emailSplitFindIndex = splitdata[i+1].find('>')
                result.append(splitdata[i+1][:emailSplitFindIndex])
                break
            if splitdata[i] == privacyIndexData[j]:
                result.append(splitdata[i+1])
    return result

def schoolinfo(splitdata):
    result = []

    schoolIndexData = ['학교명', '전공', '성적', '졸업구분', '소재지']
    for i in range(len(splitdata)):
        for j in range(len(schoolIndexData)):
            if splitdata[i] == '성적':
                if splitdata[i+5] != '':
                    score = float(splitdata[i+5].replace('/4.5', ''))
                    if score >= 4.0:
                        result.append(10)
                        break
                    elif score >= 3.7:
                        result.append(9)
                        break
                    elif score >= 3.5:
                        result.append(8)
                        break
                    elif score >= 3.0:
                        result.append(7)
                        break
                    else:
                        result.append(6)
                        break

            if splitdata[i] == '소재지':
                idx = splitdata[i+5].find('>')
                result.append(splitdata[i+5][:idx])
                break

            if splitdata[i] == schoolIndexData[j]:
                result.append(splitdata[i+5])
    return result

def toeicinfo(splitdata):
    '''
    900점 이상 10점
    900점 ~ 800점 9.5점
    800점 ~ 700점 9점
    700점 ~ 600점 8.5점
    600점 미만 4점
    '''
    result = []
    for i in range(len(splitdata)):
        if splitdata[i] == 'TOEIC':
            ## 수정
            # print(splitdata[i+5].replace('점', '') 바뀌지 않는 값이 들어감을 확인
            try:
                score = int(splitdata[i+5].replace('점', ''))
            except ValueError:
                continue
            
            if score >= 900:
                result.append(10)
                break
            elif score >= 800:
                result.append(9.5)
                break
            elif score >= 700:
                result.append(9)
                break
            elif score >= 600:
                result.append(8.5)
                break
            else:
                result.append(4)
                break
            
    return result
    
def certificateinfo(splitdata):
    '''
    기사자격증 - 4점
    그외 기타 자격증 - 1점
    만점 - 5점
    '''
    result = []
    
    idx_start = splitdata.index('점수')
    idx_end = splitdata.index('기간')
    result_origin = splitdata[idx_start+1:idx_end]
    
    del result_origin[3]
    result_origin[-1] = result_origin[-1][:result_origin[-1].find('>')]
    score = 0
    certificateName = []
    for i in range(0, len(result_origin), 3):
        if result_origin[i].find('기사') != -1:
            # print(result_origin[i])
            score += 4
        else:
            # print(result_origin[i])
            score += 1
            
    if score > 5:
        score = 5
    result.append(score)
    
    return result 
    

def educationinfo(splitdata):
    '''
    교육/활동/수상 - 1회당 2점 (가산점)
    최대 5점
    '''
    result = []
        
    교육index = [
        '군별', 
        '계급', 
        '복무기간'
    ]
    활동 = []
    for i in 교육index:
        if splitdata[splitdata.index(i)-1]:
            활동.append(splitdata[splitdata.index(i)-1])
    
    score = len(활동) * 2
    
    if score > 5:
        score = 5
        
    result.append(score)
        
    return result

def militaryinfo(splitdata):
    result = []
    군복무index = [
        '군필여부',
        '군별',
        '계급',
        '복무기간'
    ]
    
    for i in range(len(splitdata)):
        for j in range(len(군복무index)):
            if splitdata[i] == '복무기간':
                idx = splitdata[i+1].find('>')
                result.append(splitdata[i+1][:idx])
                break
            if splitdata[i] == 군복무index[j]:
                result.append(splitdata[i+1])
                
    return result

def languageinfo(splitdata):
    '''
    상 - 2점
    중 - 1.5점
    하 - 1점
    '''
    result = []
    result1 = []
    언어index = ['R', 'Python', 'Javascript', 'SQL', 'HTML/CSS']
    for i in range(len(splitdata)):
        for j in range(len(언어index)):
            if splitdata[i] == 'HTML/CSS':
                idx = splitdata[i+5].find('>')
                result1.append(splitdata[i+5][:idx])
                break
            if splitdata[i] == 언어index[j]:
                result1.append(splitdata[i+5])
    
    score = 0
    score += result1.count('상') * 2
    score += result1.count('중') * 1.5
    score += result1.count('하') * 1
    
    result.append(score)
                
    return result

functions = [
    privacyinfo, 
    schoolinfo, 
    toeicinfo, 
    certificateinfo, 
    educationinfo,
    militaryinfo,
    languageinfo
]

In [32]:
# 총 100개의 이력서.hwp 파일입니다 
import glob
import xlsxwriter
import numpy as np
import pandas as pd
import ole
path = ('./이력서/*.hwp')
workbook = xlsxwriter.Workbook('tutorial_이력서.xlsx')
worksheet = workbook.add_worksheet()
num = 1
df = ['이름', '생년월일', '주소', '번호', '이메일',
       '대학교', '전공', '학점', '졸업구분', '소재지', 'TOEIC',
      '자격증',  '교육/활동/수상', '군필여부',
       '군별', '계급', '복무기간', '컴퓨터언어능력']
for n,i in enumerate(df):
    worksheet.write(0,n ,i)  

for fname in glob.glob(path):
       with ole.open(fname) as f:
            total =[]
            data = f.get_stream('PrvText').read().decode('utf-16le')
            data = data.split('><')

            for function in functions:
                total.extend(function(data))
 
                for number,values in enumerate(total):
                    worksheet.write(num ,number, values)
            num += 1
           

workbook.close()

       

### 2. 확인용 데이터 원본 포함하고 출력하기  

In [ ]:
## ## 확인용 데이터 원본 제외하고 출력하기 

def privacyinfo(splitdata):
    result = []
    privacyIndexData = [
    '성          명',
    '생  년  월  일',
    '현    주    소',
    '연    락    처',
    '이    메    일'
     ]
    for i in range(len(splitdata)):
        for j in range(len(privacyIndexData)):
            if splitdata[i] == '이    메    일':
                emailSplitFindIndex = splitdata[i+1].find('>')
                result.append(splitdata[i+1][:emailSplitFindIndex])
                break
            if splitdata[i] == privacyIndexData[j]:
                result.append(splitdata[i+1])
    return result

def schoolinfo(splitdata):
    result = []

    schoolIndexData = ['학교명', '전공', '성적', '졸업구분', '소재지']
    for i in range(len(splitdata)):
        for j in range(len(schoolIndexData)):
            if splitdata[i] == '성적':
                if splitdata[i+5] != '':
                    score = float(splitdata[i+5].replace('/4.5', ''))
                    if score >= 4.0:
                        result.append(10)
                        break
                    elif score >= 3.7:
                        result.append(9)
                        break
                    elif score >= 3.5:
                        result.append(8)
                        break
                    elif score >= 3.0:
                        result.append(7)
                        break
                    else:
                        result.append(6)
                        break

            if splitdata[i] == '소재지':
                idx = splitdata[i+5].find('>')
                result.append(splitdata[i+5][:idx])
                break

            if splitdata[i] == schoolIndexData[j]:
                result.append(splitdata[i+5])
    return result

def toeicinfo(splitdata):
    '''
    900점 이상 10점
    900점 ~ 800점 9.5점
    800점 ~ 700점 9점
    700점 ~ 600점 8.5점
    600점 미만 4점
    '''
    result = []
    for i in range(len(splitdata)):
        if splitdata[i] == 'TOEIC':
            ## 수정
            # print(splitdata[i+5].replace('점', '') 바뀌지 않는 값이 들어감을 확인
            try:
                score = int(splitdata[i+5].replace('점', ''))
            except ValueError:
                continue
            
            if score >= 900:
                result.append(10)
                break
            elif score >= 800:
                result.append(9.5)
                break
            elif score >= 700:
                result.append(9)
                break
            elif score >= 600:
                result.append(8.5)
                break
            else:
                result.append(4)
                break
            
    return result
    
def certificateinfo(splitdata):
    '''
    기사자격증 - 4점
    그외 기타 자격증 - 1점
    만점 - 5점
    '''
    result = []
    
    idx_start = splitdata.index('점수')
    idx_end = splitdata.index('기간')
    result_origin = splitdata[idx_start+1:idx_end]
    
    del result_origin[3]
    result_origin[-1] = result_origin[-1][:result_origin[-1].find('>')]
    score = 0
    certificateName = []
    for i in range(0, len(result_origin), 3):
        if result_origin[i].find('기사') != -1:
            # print(result_origin[i])
            score += 4
        else:
            # print(result_origin[i])
            score += 1
            
    if score > 5:
        score = 5
    result.append(score)
    
    return result 
    

def educationinfo(splitdata):
    '''
    교육/활동/수상 - 1회당 2점 (가산점)
    최대 5점
    '''
    result = []
        
    교육index = [
        '군별', 
        '계급', 
        '복무기간'
    ]
    활동 = []
    for i in 교육index:
        if splitdata[splitdata.index(i)-1]:
            활동.append(splitdata[splitdata.index(i)-1])
    
    score = len(활동) * 2
    
    if score > 5:
        score = 5
        
    result.append(score)
        
    return result

def militaryinfo(splitdata):
    result = []
    군복무index = [
        '군필여부',
        '군별',
        '계급',
        '복무기간'
    ]
    
    for i in range(len(splitdata)):
        for j in range(len(군복무index)):
            if splitdata[i] == '복무기간':
                idx = splitdata[i+1].find('>')
                result.append(splitdata[i+1][:idx])
                break
            if splitdata[i] == 군복무index[j]:
                result.append(splitdata[i+1])
                
    return result

def languageinfo(splitdata):
    '''
    상 - 2점
    중 - 1.5점
    하 - 1점
    '''
    result = []
    result1 = []
    언어index = ['R', 'Python', 'Javascript', 'SQL', 'HTML/CSS']
    for i in range(len(splitdata)):
        for j in range(len(언어index)):
            if splitdata[i] == 'HTML/CSS':
                idx = splitdata[i+5].find('>')
                result1.append(splitdata[i+5][:idx])
                break
            if splitdata[i] == 언어index[j]:
                result1.append(splitdata[i+5])
    
    score = 0
    score += result1.count('상') * 2
    score += result1.count('중') * 1.5
    score += result1.count('하') * 1
    
    result.append(score)
                
    return result

functions = [
    privacyinfo, 
    schoolinfo, 
    toeicinfo, 
    certificateinfo, 
    educationinfo,
    militaryinfo,
    languageinfo
]